In [1]:
# 2) Dataset scan + build data.yaml for pose (robust scan)
from pathlib import Path
import os, yaml
from glob import glob
from collections import Counter

root = Path('./dataset')
assert root.exists(), "Dataset folder './dataset' not found. Please scp/rsync it first."

# Find YOLO label .txt anywhere under labels/ (supports both layouts)
label_txts = []
label_txts += glob(str(root / 'labels' / '**' / '*.txt'), recursive=True)          # type-first: dataset/labels/train/*.txt
label_txts += glob(str(root / '*' / 'labels' / '**' / '*.txt'), recursive=True)    # split-first: dataset/train/labels/*.txt
label_txts = sorted(set(label_txts))
print(f"Found {len(label_txts)} label files under 'labels' folders")

if len(label_txts) == 0:
    # Last resort: search any .txt and warn
    label_txts = glob(str(root / '**' / '*.txt'), recursive=True)
    print(f"Fallback: found {len(label_txts)} .txt files total")
    assert len(label_txts)>0, "No label .txt files found anywhere in './dataset'. Expected YOLO-pose labels under .../labels/..."

# Robustly infer keypoints K by majority across many labels
kp_counter = Counter()
for i, fp in enumerate(label_txts[:2000]):  # sample up to 2000 files
    try:
        with open(fp, 'r') as f:
            for j, ln in enumerate(f):
                ln = ln.strip()
                if not ln:
                    continue
                parts = ln.split()
                # prefer full pose labels: cls cx cy w h + 3*K
                if len(parts) >= 8 and (len(parts) - 5) % 3 == 0:
                    Kcand = (len(parts) - 5) // 3
                    kp_counter[Kcand] += 1
                # stop early per file after a couple of lines
                if j >= 2:
                    break
    except Exception:
        continue

if not kp_counter:
    # Fallback: detect K from kpt-only (cls x1 y1 x2 y2 ...)
    kpt_only_counter = Counter()
    for i, fp in enumerate(label_txts[:2000]):
        try:
            with open(fp, 'r') as f:
                for j, ln in enumerate(f):
                    ln = ln.strip()
                    if not ln:
                        continue
                    parts = ln.split()
                    if len(parts) >= 3 and (len(parts) - 1) % 2 == 0:
                        Kcand = (len(parts) - 1) // 2
                        kpt_only_counter[Kcand] += 1
                    if j >= 2:
                        break
        except Exception:
            continue
    assert kpt_only_counter, 'Could not infer keypoints K from labels.'
    K = max(kpt_only_counter, key=kpt_only_counter.get)
else:
    K = max(kp_counter, key=kp_counter.get)

print(f"Inferred keypoints K (majority) = {K}")

# Heuristics to resolve images dirs for each split
def choose_images_dir(split: str):
    candidates = [
        root / 'images' / split,       # type-first
        root / split / 'images',       # split-first
    ]
    # derive from labels/<split>
    split_label_dirs = []
    for p in label_txts:
        p_norm = p.replace('\\', '/').lower()
        if f"/labels/{split}/" in p_norm:
            split_label_dirs.append(Path(p).parent)
    if split_label_dirs:
        ldir = split_label_dirs[0]
        candidates += [
            ldir.parent / 'images',                  # .../train/images
            ldir.parent.parent / 'images' / ldir.name # .../images/train
        ]
    for c in candidates:
        if c.exists():
            return str(c)
    return None

train_path = choose_images_dir('train')
val_path = choose_images_dir('val')
test_path = choose_images_dir('test')
print('Images dirs (raw):', {'train': train_path, 'val': val_path, 'test': test_path})

# Normalize to be RELATIVE to dataset root if possible, else absolute
root_abs = root.resolve()

def to_rel_or_abs(p):
    if not p:
        return None
    p_abs = Path(p)
    try:
        p_abs = p_abs.resolve()
    except Exception:
        p_abs = (root / p).resolve()
    try:
        return str(p_abs.relative_to(root_abs))
    except ValueError:
        return str(p_abs)

train_rel = to_rel_or_abs(train_path)
val_rel = to_rel_or_abs(val_path)
test_rel = to_rel_or_abs(test_path)
print('Images dirs (normalized):', {'train': train_rel, 'val': val_rel, 'test': test_rel})

assert train_rel and val_rel, (
    "Could not locate images/train and/or images/val. Ensure YOLO structure like:\n"
    "- dataset/images/train & dataset/labels/train\n"
    "- dataset/train/images & dataset/train/labels\n"
)

# Build YAML for pose
names = ['plate']
skeleton = [[0,1],[1,2],[2,3],[3,0]] if K==4 else []
flip_idx = list(range(K))

data = {
    'path': str(root_abs),
    'train': train_rel,
    'val': val_rel,
    'test': test_rel,
    'names': names,
    'kpt_shape': [K, 3],
    'skeleton': skeleton,
    'flip_idx': flip_idx,
}

yaml_path = root/'data.yaml'
with open(yaml_path, 'w') as f:
    yaml.safe_dump(data, f, sort_keys=False)

print('Wrote', yaml_path)
print(yaml.safe_dump(data, sort_keys=False))

# Clear label caches so Ultralytics rebuilds with the corrected kpt_shape
for cache_name in ['train.cache', 'val.cache']:
    cp = root / 'labels' / cache_name
    if cp.exists():
        try:
            os.remove(cp)
            print('Removed cache:', cp)
        except Exception as e:
            print('Failed to remove cache', cp, e)


Found 12809 label files under 'labels' folders
Inferred keypoints K (majority) = 2
Images dirs (raw): {'train': 'dataset/images/train', 'val': 'dataset/images/val', 'test': None}
Images dirs (normalized): {'train': 'images/train', 'val': 'images/val', 'test': None}
Wrote dataset/data.yaml
path: /home/azazel/creation/license_plate_recognition/dataset
train: images/train
val: images/val
test: null
names:
- plate
kpt_shape:
- 2
- 3
skeleton: []
flip_idx:
- 0
- 1

Removed cache: dataset/labels/train.cache
Removed cache: dataset/labels/val.cache


In [ ]:
# 2.b) Merge extra datasets (datasets2 + dataset3/LP_detection) into ./dataset
from pathlib import Path
import shutil, os, glob

root_main = Path('./dataset')
root_ds2 = Path('./datasets2')
root_ds3 = Path('./dataset3/LP_detection')  # use LP_detection split in dataset3

assert root_main.exists(), "./dataset must exist (target)"
print('Target dataset:', root_main.resolve())

# Ensure subfolders
for p in [root_main/'images'/ 'train', root_main/'images'/'val', root_main/'labels'/'train', root_main/'labels'/'val']:
    p.mkdir(parents=True, exist_ok=True)

# Helper to copy tree contents (images + labels) preserving names
IMG_EXTS = {'.jpg','.jpeg','.png','.bmp','.tif','.tiff','.webp'}

def copy_split(src_images: Path, src_labels: Path, dst_images: Path, dst_labels: Path):
    if src_images and src_images.exists():
        files = [p for p in src_images.rglob('*') if p.suffix.lower() in IMG_EXTS]
        print(f'Copying {len(files)} images from', src_images)
        for fp in files:
            rel = fp.name
            shutil.copy2(fp, dst_images/rel)
    if src_labels and src_labels.exists():
        files = list(src_labels.rglob('*.txt'))
        print(f'Copying {len(files)} labels from', src_labels)
        for fp in files:
            rel = fp.name
            shutil.copy2(fp, dst_labels/rel)

# Merge from datasets2 (assumes YOLO layout)
if root_ds2.exists():
    copy_split(root_ds2/'images'/'train', root_ds2/'labels'/'train', root_main/'images'/'train', root_main/'labels'/'train')
    copy_split(root_ds2/'images'/'val',   root_ds2/'labels'/'val',   root_main/'images'/'val',   root_main/'labels'/'val')
else:
    print('datasets2 not found, skip')

# Merge from dataset3/LP_detection (assumes YOLO layout)
if root_ds3.exists():
    copy_split(root_ds3/'images'/'train', root_ds3/'labels'/'train', root_main/'images'/'train', root_main/'labels'/'train')
    copy_split(root_ds3/'images'/'val',   root_ds3/'labels'/'val',   root_main/'images'/'val',   root_main/'labels'/'val')
else:
    print('dataset3/LP_detection not found, skip')

# Clear caches so Ultralytics reindexes
for cache_name in ['train.cache', 'val.cache']:
    cp = root_main / 'labels' / cache_name
    if cp.exists():
        try:
            os.remove(cp)
            print('Removed cache:', cp)
        except Exception as e:
            print('Failed to remove cache', cp, e)

print('Merge done. Now re-run the data.yaml build cell above to re-assert paths if needed.')


In [4]:
# 3) Train (quiet + low-lag)
from ultralytics import YOLO
from pathlib import Path
import torch, os, logging

# Reduce notebook lag/noise
os.environ.setdefault('TQDM_DISABLE', '1')  # disable tqdm progress bars
os.environ.setdefault('ULTRALYTICS_HUB', '0')
os.environ.setdefault('WANDB_DISABLED', 'true')
try:
    from ultralytics.utils import LOGGER
    LOGGER.setLevel(logging.ERROR)  # minimal logs
except Exception:
    pass

# Use local data.yaml and local best.pt only
yaml_path = Path('./dataset/data.yaml')
assert yaml_path.exists(), "dataset/data.yaml not found. Run the dataset cell first."

ckpt = str(Path('best.pt').resolve())
assert os.path.exists(ckpt), "best.pt not found at project root. Place your checkpoint at ./best.pt"

print(f'Training (quiet) from: {ckpt}')
model = YOLO(ckpt)

# Minimal, low-I/O training args
train_args = dict(
    data=str(yaml_path),
    imgsz=640,        # smaller image reduces memory/compute
    epochs=200,       # adjust as needed
    patience=10,      # early stopping
    batch=4,          # tune for your VRAM
    device=0 if torch.cuda.is_available() else 'cpu',
    workers=int(os.getenv('YOLO_WORKERS', '4')),
    rect=True,
    cache=os.getenv('YOLO_CACHE', 'disk'),  # use 'disk' to save RAM
    pretrained=False,  # do not download any pretrained weights
    plots=False,       # no plot generation
    save_period=0,     # don't save intermediate epochs
    val=False,         # skip per-epoch validation to reduce overhead
    verbose=False,     # quiet logs
    project='runs',
    name='pose_plate',
    exist_ok=True,
    seed=42,
)

results = model.train(**train_args)
try:
    save_dir = getattr(getattr(model, 'trainer', None), 'save_dir', None)
    if save_dir:
        print('Training done. Results saved to:', save_dir)
    else:
        print('Training done. Check runs/pose_plate for results.')
except Exception:
    print('Training done.')

Training (quiet) from: /home/azazel/creation/license_plate_recognition/best.pt


RuntimeError: Ultralytics offline: asset not found locally: yolo11n.pt

In [ ]:
# 4) Validate & quick inference
from glob import glob
import os, yaml
import cv2, numpy as np

# Strict offline mode: allow local assets but block any remote downloads
os.environ.setdefault('ULTRALYTICS_HUB', '0')
os.environ.setdefault('WANDB_DISABLED', 'true')
try:
    from pathlib import Path as _P
    from ultralytics.utils import downloads as _ud
    from ultralytics.utils import checks as _uc
    _ud.is_online = lambda: False  # type: ignore

    def _local_only(asset, *args, **kwargs):
        # Return the local path if it exists; otherwise block
        try:
            p = _P(asset)
        except TypeError:
            try:
                p = _P(asset[0])
            except Exception:
                p = None
        if p is not None and p.exists():
            return str(p)
        raise RuntimeError(f"Ultralytics offline: asset not found locally: {asset}")

    for _name in ("safe_download", "attempt_download", "attempt_download_asset", "get_github_assets"):
        if hasattr(_ud, _name):
            setattr(_ud, _name, _local_only)
    if hasattr(_uc, "check_requirements"):
        _uc.check_requirements = lambda *a, **k: None  # type: ignore
except Exception:
    pass

from ultralytics import YOLO
from pathlib import Path

# Force local checkpoint only (absolute path)
ckpt = str(Path('best.pt').resolve())
assert os.path.exists(ckpt), "Local checkpoint './best.pt' not found. Place your trained weights as best.pt at project root."
print('Loading weights:', ckpt)
model = YOLO(ckpt)
print('Model task:', getattr(model, 'task', None))

# read data.yaml to find val images path
with open('./dataset/data.yaml', 'r') as f:
    data_cfg = yaml.safe_load(f)
val_path = data_cfg.get('val')
base_path = data_cfg.get('path', '.')
val_dir = os.path.join(base_path, val_path) if val_path else './dataset/val/images'

# pick a few images
val_imgs = []
for pat in ['*.jpg', '*.jpeg', '*.png', '*.*']:
    val_imgs = glob(os.path.join(val_dir, pat))
    if val_imgs:
        break
val_imgs = val_imgs[:6]
print('Previewing', len(val_imgs), 'images from', val_dir)


def _to_numpy(x):
    try:
        import torch
        if isinstance(x, np.ndarray):
            return x
        if isinstance(x, torch.Tensor):
            return x.detach().cpu().numpy()
    except Exception:
        pass
    return np.asarray(x)


def _order_points_four(pts: np.ndarray) -> np.ndarray:
    pts = np.asarray(pts, dtype='float32')
    if pts.shape[0] != 4:
        return pts
    s = pts.sum(axis=1)
    diff = pts[:, 1] - pts[:, 0]
    tl = pts[np.argmin(s)]
    br = pts[np.argmax(s)]
    tr = pts[np.argmin(diff)]
    bl = pts[np.argmax(diff)]
    return np.array([tl, tr, br, bl], dtype='float32')


for imgp in val_imgs:
    im = cv2.imread(imgp)
    res = model.predict(source=im, imgsz=640, conf=0.5, iou=0.6, classes=[0], verbose=False, max_det=50)[0]

    # Draw boxes
    if res.boxes is not None and len(res.boxes) > 0:
        for b in res.boxes:
            x1,y1,x2,y2 = b.xyxy[0].int().cpu().tolist()
            cv2.rectangle(im, (x1,y1), (x2,y2), (0,255,0), 2)

    # Draw keypoints with CPU conversion and polygon
    if getattr(res, 'keypoints', None) is not None and res.keypoints.xy is not None:
        kxy = _to_numpy(res.keypoints.xy)
        kcf = _to_numpy(getattr(res.keypoints, 'conf', None)) if getattr(res.keypoints, 'conf', None) is not None else None
        for i in range(kxy.shape[0]):
            pts = kxy[i]
            if pts is None or pts.shape[0] < 4:
                continue
            mask = np.ones((pts.shape[0],), dtype=bool)
            if kcf is not None:
                mask = kcf[i] >= 0.1
            pts_vis = pts[mask]
            if pts_vis.shape[0] >= 4:
                pts4 = pts_vis[:4]
                ordered = _order_points_four(pts4)
                poly = ordered.reshape((-1,1,2)).astype(int)
                cv2.polylines(im, [poly], isClosed=True, color=(0,0,255), thickness=2)
                for px, py in ordered:
                    cv2.circle(im, (int(px), int(py)), 4, (0,255,255), -1)
            else:
                for px, py in pts[:4]:
                    cv2.circle(im, (int(px), int(py)), 4, (0,255,255), -1)

    cv2.imshow('preview', im)
    if cv2.waitKey(0) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()